In [1]:
import time
import numpy as np
import gymnasium as gym
import torch
from gymnasium import spaces
from gymnasium.spaces import Box
from pystk2_gymnasium import AgentSpec
from pystk2_gymnasium import FlattenerWrapper, PolarObservations, ConstantSizedObservations, MonoAgentWrapperAdapter
from pystk2_gymnasium.stk_wrappers import OnlyContinuousActionsWrapper
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_checker import check_env

2024-11-10 19:00:04.432068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731261604.461370   15053 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731261604.469762   15053 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 19:00:04.493547: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# On teste l'environnement avec des actions random
# Normalement si t'es sur WSL le render_mode = 'human' va te causer pb vu qu'il y a pas d'interface graphique. 
# Faut faire un X11 forwarding, perso j'ai DL XLaunch pour faire ça.

env = gym.make("supertuxkart/full-v0", agent=AgentSpec(use_ai=False,name="Hatem"),render_mode=None, track='abyss',max_paths=5)
ix = 0
done = False
state, *_ = env.reset() 

while not done:
    ix += 1
    action = env.action_space.sample()
    print(action)
    state, reward, terminated, truncated, _ = env.step(action)
    print(env.observation_space,state)
    done = truncated or terminated
    break
    

..:: Antarctica Rendering Engine 2.0 ::..
OrderedDict([('acceleration', array([0.19325757], dtype=float32)), ('brake', np.int64(0)), ('drift', np.int64(0)), ('fire', np.int64(0)), ('nitro', np.int64(1)), ('rescue', np.int64(0)), ('steer', array([0.02735091], dtype=float32))])
Dict('attachment': Discrete(10), 'attachment_time_left': Box(0.0, inf, (1,), float32), 'center_path': Box(-inf, inf, (3,), float32), 'center_path_distance': Box(-inf, inf, (1,), float32), 'distance_down_track': Box(-inf, inf, (1,), float32), 'energy': Box(0.0, inf, (1,), float32), 'front': Box(-inf, inf, (3,), float32), 'items_position': Sequence(Box(-inf, inf, (3,), float32), stack=False), 'items_type': Sequence(Discrete(7), stack=False), 'jumping': Discrete(2), 'karts_position': Sequence(Box(-inf, inf, (3,), float32), stack=False), 'max_steer_angle': Box(-1.0, 1.0, (1,), float32), 'paths_distance': Sequence(Box(0.0, inf, (2,), float32), stack=False), 'paths_end': Sequence(Box(-inf, inf, (3,), float32), stack=Fal

In [22]:
# Je vais te monter le pb dont je te parlais tout à l'heure lié à pystk2_gymnasium.
# En gros, le .reset() est buggué, exemple : 

env = gym.make("supertuxkart/full-v0", agent=AgentSpec(use_ai=False,name="Hatem"),render_mode=None, track='abyss',max_paths=5)
done = False
state, *_ = env.reset() # Tout roule
action = env.action_space.sample()
env.step(action)
env.reset() # Là normalement t'as une erreur.
env.close()

# En soi normalement on s'en balek, mais le truc c'est que ce problème va faire crash le PPO de stable baseline plus bas, donc faut le fix.
# Pour faire ça, faut aller dans envs.py, et modifier reset. Voilà la modif:

# def reset(
#         self,
#         *,
#         seed: Optional[int] = None,
#         options: Optional[Dict[str, Any]] = None,
#     ) -> Tuple[pystk2.WorldState, Dict[str, Any]]:
        
#         random = np.random.RandomState(seed)

#         # Ajout d'un nettoyage manuel pour self.race
#         if self.race:
#             print("Nettoyage de la course en cours avant le reset.")
#             self.race = None  # Libère la course manuellement pour éviter les conflits

..:: Antarctica Rendering Engine 2.0 ::..


In [ ]:
class FilteredObservationWrapper(gym.ObservationWrapper):
    """
    Wrapper permettant de garder uniquement les observations présentes dans keys_to_keep
    """
    def __init__(self, env, keys_to_keep):
        super().__init__(env)
        self.keys_to_keep = keys_to_keep
        
        # Filtrage des dimensions et transformation en espace Box
        low = []
        high = []
        for key in keys_to_keep:
            space = self.env.observation_space[key]
            low.extend(space.low.flatten())
            high.extend(space.high.flatten())
        
        # Définir le nouvel espace d'observation comme un Box 1D
        self.observation_space = spaces.Box(low=np.array(low), high=np.array(high), dtype=np.float32)

    def observation(self, observation):
        # Filtrage des clés et conversion en vecteur 1D
        filtered_values = [observation[key].flatten() for key in self.keys_to_keep]
        return np.concatenate(filtered_values)

# Clés à garder dans l'observation (modifiez selon vos préférences)
keys_to_keep = [
    'velocity',            # Vitesse actuelle du kart
    'center_path_distance',# Distance du chemin
    'max_steer_angle',     # Angle maximum de direction
    'front',
    'center_path',
    'paths_start', # Empiriquement, ces 5 là changent radicalement l'entraînement : On passe d'un bot qui tombe partout à un truc vraiment correct grâce
    'paths_width', # à eux. 
    'paths_end'
    
]

keys_to_keep2 = ['center_path', 'center_path_distance', 'distance_down_track', 'energy', 'front', 
                 'karts_position', 'max_steer_angle', 'paths_distance', 'paths_end', 'paths_start', 
                'paths_width', 'skeed_factor', 'velocity']

all_obs_keys=['attachment', 'attachment_time_left', 'center_path', 'center_path_distance', 'distance_down_track', 'energy', 'front', 
              'items_position', 'items_type', 'jumping', 'karts_position', 'max_steer_angle', 'paths_distance', 'paths_end', 'paths_start',
              'paths_width', 'powerup', 'shield_time', 'skeed_factor', 'velocity']

In [ ]:
class DictToBoxActionWrapper(gym.ActionWrapper):
    """
    Wrapper permettant de passer d'un dictionnaire d'actions à des actions sous forme matricielle pour PPO après
    """
    def __init__(self, env):
        super(DictToBoxActionWrapper, self).__init__(env)
        # Définir l'espace d'action en tant que Box 1D qui combine l'accélération et la direction
        self.action_space = Box(low=np.array([0.0, -1.0]), high=np.array([1.0, 1.0]), dtype=np.float32)

    def action(self, action):
        # Diviser l'action en 'acceleration' et 'steer' à partir du vecteur 1D
        return {'acceleration': np.array([action[0]], dtype=np.float32), 'steer': np.array([action[1]], dtype=np.float32)}

In [25]:
class DictToBoxActionWrapper(gym.ActionWrapper):
    """
    Wrapper qui convertit un dictionnaire d'actions en un vecteur d'actions pour PPO.
    Il combine toutes les actions discrètes et continues en une seule Box 1D.
    """
    def __init__(self, env):
        super(DictToBoxActionWrapper, self).__init__(env)
        # Définir l'espace d'action combiné : [acceleration, steer, brake, drift, fire, nitro, rescue]
        # acceleration et steer sont continues, les autres sont discrètes (0 ou 1) mais on les représente en float32
        self.action_space = Box(
            low=np.array([0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0]), 
            high=np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]), 
            dtype=np.float32
        )

    def action(self, action):
        """
        Convertit l'action 1D en dictionnaire d'actions pour l'environnement original.
        """
        # Extraire chaque composant d'action à partir du vecteur d'entrée
        action = {
            'acceleration': np.array([action[0]], dtype=np.float32),
            'steer': np.array([action[1]], dtype=np.float32),
            'brake': 0,    # 0 ou 1
            'drift': int(action[3] > 0.5),    # 0 ou 1
            'fire': int(action[4] > 0.5),     # 0 ou 1
            'nitro': int(action[5] > 0.5),    # 0 ou 1
            'rescue': 0   # 0 ou 1
        }
        return action

In [ ]:
if not torch.cuda.is_available():
    print("Attention : CUDA n'est pas disponible. Le modèle sera entraîné sur CPU.")
else:
    print("CUDA est disponible. L'entraînement se fera sur GPU.")

env = gym.make("supertuxkart/full-v0", agent=AgentSpec(use_ai=False, name="Hatem"),render_mode=None,track='abyss')
# Entraînement uniquement sur la map 'abyss' pour le moment
env = PolarObservations(ConstantSizedObservations(env))
env = FilteredObservationWrapper(env, keys_to_keep)
env = DictToBoxActionWrapper(env)

# En gros, ConstantSizedObservations permet de passer à un environnement "réduit", voir la doc en vrai c'est bien expliqué
# PolarObservations ajoute des observations utiles (passage en coordonnées polaire pour avoir max_steer_angle notamment)

check_env(env,warn=True) # Permet de check si tout est en ordre

# Configuration et lancement de PPO
model = PPO(
    "MlpPolicy",    # Utilise un réseau de neurones entièrement connecté (MLP) comme politique
    env,            
    verbose=1,      # Active les messages de log pour suivre l'entraînement
    n_steps=2048,   # Nombre d'étapes dans chaque mise à jour de PPO
    batch_size=64,  # Taille des batchs
    gae_lambda=0.95, # Paramètre pour l'estimation de l'avantage généralisé
    gamma=0.99,     
    ent_coef=0.01,  
    learning_rate=2.5e-4, 
    device="cuda" 
)

model.learn(total_timesteps=40000)

#model.save("../models/ppo_supertuxkart_2")

Attention : CUDA n'est pas disponible. Le modèle sera entraîné sur CPU.
..:: Antarctica Rendering Engine 2.0 ::..
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1.5e+03  |
|    ep_rew_mean     | -132     |
| time/              |          |
|    fps             | 132      |
|    iterations      | 1        |
|    time_elapsed    | 15       |
|    total_timesteps | 2048     |
---------------------------------


KeyboardInterrupt: 

In [9]:
model = PPO.load("../models/ppo_supertuxkart_full_act")

env = gym.make("supertuxkart/full-v0", agent=AgentSpec(use_ai=False, name="Hatem"),render_mode="human",track='abyss')
env = PolarObservations(ConstantSizedObservations(env))
env = FilteredObservationWrapper(env, keys_to_keep)
env = DictToBoxActionWrapper(env)

state,_ = env.reset()
done = False    
while not done:
    action, _ = model.predict(state, deterministic=True)
    state, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

env.close()

..:: Antarctica Rendering Engine 2.0 ::..


KeyError: 'brake'

## Ajout d'un Wrapper pour le Turbo en début de partie

In [2]:
class TurboStartWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.in_countdown = True
        self.current_step = 0

    def action(self, action):
        # Pendant la phase de décompte, applique une accélération continue pour obtenir le boost
        if self.in_countdown:
            action["acceleration"] = np.array([1.0], dtype='float32')
            action["steer"] = np.array([0], dtype='float32')
            self.current_step += 1

            # Conditions de fin du compte à rebours (exemple : après 3 étapes, modifiez selon vos observations)
            if self.current_step >= 200:
                self.in_countdown = False

        # Exécute l'étape avec l'action, et passe au mode normal après le décompte

        return action
        


In [24]:
from collections import OrderedDict

# Visualisation pour voir si ça marche bien

env = gym.make("supertuxkart/full-v0", agent=AgentSpec(use_ai=False,name="Hatem"),render_mode='human', track='abyss',max_paths=5)
# env = TurboStartWrapper(OnlyContinuousActionsWrapper(PolarObservations(ConstantSizedObservations(env))))
env = DictToBoxActionWrapper(env)
done = False
print("uh")
state, *_ = env.reset() 
print("aah")
ix = 0

while not done:
    ix+=1
    # action = OrderedDict([('acceleration', np.array([1.], dtype=np.float32)),('steer', np.array([0.], dtype=np.float32))])
    action = env.action_space.sample()
    print(env.action_space,action)
    state, reward, terminated, truncated, _ = env.step(action)
    done = truncated or terminated
    if ix%150==0:
        break

env.close()

uh
..:: Antarctica Rendering Engine 2.0 ::..
aah
Box([ 0. -1.  0.  0.  0.  0.  0.], 1.0, (7,), float32) [ 0.79164255 -0.5695906   0.6757963   0.732773    0.5854332   0.654038
  0.67417455]
WTFFFFF
[ 0.79164255 -0.5695906   0.6757963   0.732773    0.5854332   0.654038
  0.67417455]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [3]:
print(*{"key":0,"hey":1})

key hey
